In [1]:
import os
os.chdir("../../")
import math
import re
import numpy as np
import pandas as pd
import pdfplumber
import tabula
import PyPDF2
import warnings
warnings.filterwarnings("ignore")

In [2]:
pdf_path = os.getcwd() + "/data/tourism/vanuatu/2013-TM-06-June_News.pdf"

## Tonga 
### Table 1

In [4]:
tonga_lsts = os.listdir("data/tourism/tonga")
filepaths = [os.getcwd() + "/data/tourism/tonga/" +
             path for path in tonga_lsts if "Dec" in path]

In [81]:
def locate_table(filepath: str,
                 search_string: str,
                 ignore_case=False):

    search_lst = list()
    reader = PyPDF2.PdfReader(filepath)

    for page_num, page in enumerate(reader.pages):
        try:
            page_text = page.extract_text()
            hits = None
            if ignore_case == False:
                hits = re.search(search_string, page_text.lower())
            else:
                hits = re.search(
                    search_string, page_text.lower(), re.IGNORECASE)

            if hits:
                search_lst.append(page_num+1)
        except:
            pass
    return {"table_loc": search_lst}


def load_pdf(filepath: str,
             search_string: str,
             table_page: int,
             table_seq=0):

    table_loc = locate_table(filepath, search_string,
                             ignore_case=True)["table_loc"]
    if len(table_loc) != 0:
        table_page = table_loc[-1]
        dfs = tabula.read_pdf(filepath, pages=table_page, stream=True)
        if len(dfs) > 1:
            print(f"The page has {len(dfs)} tables.")
            df = dfs[table_seq]

        else:
            df = dfs[0]
            df.columns = df.iloc[0, :].to_list()
    else:
        dfs = tabula.read_pdf(filepath, pages="all", stream=True)
        df = dfs[table_page]
        df.columns = df.iloc[0, :].to_list()

    df = df.iloc[1:].reset_index().drop("index", axis=1)

    return df


def split_time(df: pd.DataFrame,
               time_var: str):

    year_idx, month_idx = list(), list()
    for idx in df.index:
        if (str(df[time_var][idx]).isdigit() == True):
            year_idx.append(idx)
        else:
            month_idx.append(idx)

    latest_year_idx = max(year_idx)

    return latest_year_idx, year_idx, month_idx


def detect_year(series: pd.Series):
    nacheck = pd.isna(series)
    start_year = int(series[nacheck == False][0])
    return start_year


def generate_time(df: pd.DataFrame,
                  start_year: int):

    years = [start_year + idx // 12 for idx in df.index]
    df["Year"] = years

    return df


def remove_separator(df: pd.DataFrame):

    colnames = df.columns
    for col in colnames:
        try:
            if df[col].dtype == "O":
                df[col] = (df[col].str.replace(",", "")
                                  .str.replace("-", "")
                                  .str.replace("(", "")
                                  .str.replace(")", "")
                                  .str.replace(" ", ""))
        except:
            print(col, "might have an error.")

    return df


def separate_data(df: pd.DataFrame,
                  var: str,
                  split_rule: str):

    splited_lst = var.split(split_rule)
    var_number = len(splited_lst)

    obj = dict()
    for i in range(var_number):
        obj[str(splited_lst[i])] = []

    for i in df[var]:
        elems = i.split(" ")
        length = len(elems)
        if length == var_number:
            idx, var = 0, list(obj.keys())
            while idx < length:
                key, val = var[idx], elems[idx]
                obj[key].append(val)
                idx += 1

        elif length < var_number:
            idx, var = 0, list(obj.keys())
            while idx < length and len(elems) != 0:
                key, val = var[idx], elems[idx].split(" ")[0]
                obj[key].append(val)
                elems = i.replace(val, "").strip()
                idx += 1
            else:
                key, val = var[idx], 0
                obj[key].append(val)
                idx += 1

        else:
            idx, var = 0, list(obj.keys())
            while idx < var_number:
                key, val = var[idx], elems[idx]
                obj[key].append(val)
                idx += 1
            else:
                key, val = var[-1], elems[idx]
                prev_val = obj[key][-1]
                obj[key][-1] = prev_val + val

    for i in range(var_number):
        df[str(splited_lst[i])] = obj[list(obj.keys())[i]]

    return df


def check_quality(df: pd.DataFrame,
                  exclude_vars: list,
                  sum_var: str):

    new_df = df.iloc[:, ~df.columns.isin(exclude_vars)]
    checked_vars = new_df.columns[~new_df.columns.isin([sum_var])].to_list()

    for idx in new_df.index:
        row_sum = 0
        for var in checked_vars:
            val = new_df[var][idx]
            if math.isnan(float(val)) != True:
                row_sum += float(val)
            else:
                row_sum += 0
        if float(new_df[sum_var][idx]) == row_sum:
            pass
        else:
            return False

    return True

In [7]:
months = pd.DataFrame()

for file in filepaths[:-1]:
    print(file)

    df = load_pdf(file, "Monthly Arrival and Departure", table_page=-5)
    latest_year, year_idx, month_idx = split_time(df, "Period")
    month = df.iloc[month_idx, 0:4]
    start_year = detect_year(df.iloc[month_idx].iloc[0])

    month = (month.dropna(how="all").reset_index()
             .drop("index", axis=1))

    print(f"The file starts from {start_year}.")

    month = separate_data(month, "Air Ship", " ").drop("Air Ship", axis=1)
    month = remove_separator(month)
    month = month.replace(r'^\s*$', 0, regex=True)

    if check_quality(month, ["Period", "Year"], "Total") == False:
        name = file.split("/")[-1].split(".")[0]
        print("  ", name, "could go wrong!")

    generate_time(month, start_year)
    print(month.head(5))
    months = pd.concat([months, month], axis=0)
    
months = (months[["Year", "Period", "Air", "Ship", "Yacht", "Total"]]
          .drop_duplicates()
          .sort_values(by="Year")
          .reset_index()
          .drop("index", axis=1))

months

/Users/czhang/Desktop/PO-Tourism/data/tonga/Migration-Report-Dec-2017.pdf
The file starts from 2013.
     Period Yacht Total   Air  Ship  Year
0   January     9  3858  2927   922  2013
1  February     2  3933  2621  1310  2013
2     March    11  3434  2797   626  2013
3     April    50  6343  3088  3205  2013
4       May   328  4293  3965     0  2013
/Users/czhang/Desktop/PO-Tourism/data/tonga/Migration-December-Report-2019.pdf
The file starts from 2018.
   Migration-December-Report-2019 could go wrong!
  Period Yacht Total   Air  Ship  Year
0    Jan     7  7640  4294  3346  2018
1    Feb     0  3856  2336  1520  2018
2    Mar    14  5911  2835  3062  2018
3    Apr    34  5477  3099  2344  2018
4    May   166  5141  4348   627  2018
/Users/czhang/Desktop/PO-Tourism/data/tonga/12-December-Migration-Report-2014.pdf


Got stderr: Nov 21, 2022 5:06:57 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Nov 21, 2022 5:06:57 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
     Period Yacht Total   Air  Ship  Year
0   January     4  3808  3158   646  2010
1  February     5  2384  2379     0  2010
2     March     5  3992  3134   853  2010
3     April    30  5650  2818  2802  2010
4       May   177  9150  3670  5303  2010
/Users/czhang/Desktop/PO-Tourism/data/tonga/12-December-Migration-Report-2015.pdf


Got stderr: Nov 21, 2022 5:07:04 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Nov 21, 2022 5:07:04 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
     Period Yacht Total   Air  Ship  Year
0   January     4  3808  3158   646  2010
1  February     5  2384  2379     0  2010
2     March     5  3992  3134   853  2010
3     April    30  5650  2818  2802  2010
4       May   177  9150  3670  5303  2010
/Users/czhang/Desktop/PO-Tourism/data/tonga/Migration-December-Report-2020.pdf
The file starts from 2018.
   Migration-December-Report-2020 could go wrong!
  Period Yacht Total   Air  Ship  Year
0    Jan     7  7640  4294  3346  2018
1    Feb     0  3856  2336  1520  2018
2    Mar    14  5911  2835  3062  2018
3    Apr    34  5477  3099  2344  2018
4    May   166  5141  4348   627  2018
/Users/czhang/Desktop/PO-Tourism/data/tonga/12-December-Migration-Report-2013.pdf


Got stderr: Nov 21, 2022 5:07:15 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Nov 21, 2022 5:07:15 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
     Period Yacht Total   Air  Ship  Year
0   January     4  3808  3158   646  2010
1  February     5  2384  2379     0  2010
2     March     5  3992  3134   853  2010
3     April    30  5650  2818  2802  2010
4       May   177  9150  3670  5303  2010
/Users/czhang/Desktop/PO-Tourism/data/tonga/12-Migration-Report-Dec-2016.pdf


Got stderr: Nov 21, 2022 5:07:23 PM org.apache.pdfbox.pdmodel.PDDocument importPage
Nov 21, 2022 5:07:23 PM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2011.
     Period Yacht Total   Air  Ship  Year
0   January     2  2846  2844   NaN  2011
1  February    14  9163  2302  6847  2011
2     March    16  3900  2477  1407  2011
3     April    37  6595  3520  3038  2011
4       May   250  4062  3812     0  2011


,Year,Period,Air,Ship,Yacht,Total
0,2010,January,3158,646,4,3808
1,2010,December,6493,0,13,6506
2,2010,November,3569,1386,126,5081
3,2010,October,3467,0,352,3819
4,2010,August,4216,0,246,4462
...,...,...,...,...,...,...
136,2020,Apr,0,0,0,0
137,2020,Mar,1259,0,0,1259
138,2020,Feb,3008,1678,6,4692
139,2020,Jul,1,0,0,1


## Vanuatu

In [25]:
pdf_path = os.getcwd() + "/data/tourism/vanuatu/2014-TM-12-December-News.pdf"

### Visitor Arrivals by Purpose of Visit

In [37]:
df = load_pdf(pdf_path, "Visitor Arrivals by Purpose of Visit", 6)
df.columns = df.iloc[0]

df = df.dropna(thresh=4, axis=1).replace("-", 0)
df = df.iloc[3:].reset_index().drop("index", axis=1)

splited = df["Conferences Stop Over"].str.split(" ", n=1, expand=True)
splited.columns = ["Conference", "Stopover"]
df = pd.concat([df, splited], axis=1)
df = remove_separator(df)

In [40]:
vu_lsts = os.listdir("data/tourism/vanuatu")
dec_lst = [file for file in vu_lsts if "Dec" in file]

In [49]:
error_dict = {
    "file": [],
    "reason": []
}


for file in dec_lst:
    if ".pdf" in file and "2010" not in file:

        print(f"{file} has started")
        filepath = os.getcwd() + "/data/tourism/vanuatu/" + file

        df = load_pdf(filepath, "Visitor Arrivals by Purpose of Visit", 6)
        df.columns = df.iloc[0]

        df = df.dropna(thresh=4, axis=1).replace("-", 0)
        df = df.iloc[3:].reset_index().drop("index", axis=1)

        try:
            col_lst = df.columns.to_list()
            stored_splited = ["Business, Stop",
                              "Cruiseship Other", "Conferences Stop Over"]

            for idx, val in enumerate(col_lst):
                if type(val) == str and val in stored_splited:
                    if val == "Business, Stop":
                        separate_data(df, "Business, Stop", ",")

                    elif val == "Conferences Stop Over":
                        splited = df[val].str.split(" ", n=1, expand=True)

                        if len(splited.columns) == 2:
                            splited.columns = [
                                val.split(" ")[0], val.split(" ")[-1]]
                            df = pd.concat([df, splited], axis=1)

                        else:
                            print(f"{file} has incompatible column.")
                            error_dict["file"].append(file)
                            error_dict["reason"].append("Incompatible Column")

                    else:
                        splited = df[val].str.split(" ", n=2, expand=True)

                        if len(splited.columns) == 2:
                            splited.columns = val.split(" ")
                            df = pd.concat([df, splited], axis=1)
                        else:
                            print("Incompatible Column")
                            error_dict["file"].append(file)
                            error_dict["reason"].append("Incompatible Column")

            df = remove_separator(df)

            try:
                df = df.drop(["Conferences Stop Over",
                             "Année", "Mois"], axis=1)

                if "Holidays" in df.columns:
                    df["Holidays"] = df["Holidays"].str.replace(" ", "")
                    saved_path = os.getcwd() + "/data/tourism/vanuatu/temp/" + \
                        file.split(".")[0] + ".csv"
                    df.to_csv(saved_path, encoding="utf-8")

                else:
                    print("  Holidays column not found.")
                    error_dict["file"].append(file)
                    error_dict["reason"].append("Holidays column not found.")

            except:
                print(f"  {file} does not find Année or Mois column.")
                error_dict["file"].append(file)
                error_dict["reason"].append("Année or Mois column not found.")

        except:
            error_dict["file"].append(file)
            error_dict["reason"].append("Column Error")

Tou12_December_News_2005.pdf has started
nan might have an error.
nan might have an error.
Other might have an error.
nan might have an error.
nan might have an error.
Other might have an error.
  Tou12_December_News_2005.pdf does not find Année or Mois column.
Tou12_December_News_2004.pdf has started
nan might have an error.
nan might have an error.
Other might have an error.
nan might have an error.
nan might have an error.
Other might have an error.
  Tou12_December_News_2004.pdf does not find Année or Mois column.
Tou12_December_News_2007.pdf has started
nan might have an error.
nan might have an error.
nan might have an error.
nan might have an error.
  Tou12_December_News_2007.pdf does not find Année or Mois column.
IAS_12_December_2015.pdf has started
2012-TM-12-December_News.pdf has started
  2012-TM-12-December_News.pdf does not find Année or Mois column.
IAS_12_December_2016.pdf has started
IVA_12_December_2021.pdf has started
2014-TM-12-December-News.pdf has started
2011-TM-

In [83]:
check_lst = os.listdir(os.getcwd() + "/data/tourism/vanuatu/temp")
check_lst = [os.getcwd() + "/data/tourism/vanuatu/temp/" + file for file in check_lst]

for file in check_lst:
    if ".DS_Store" not in file:
        df = pd.read_csv(file).drop("Unnamed: 0", axis=1)
        df = remove_separator(df)
        if check_quality(df, ["Year", "Month"], "Visitors"):
            df.to_csv(file, encoding="")
        else:
            print(f"{file} fails to pass the quality check.")

/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/IVA_12_December-English_2019.csv passed the quality check.
/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/IAS_12_December_2018.csv passed the quality check.
/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/IAS_12_Dececember_2017.csv passed the quality check.
/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/IVA_12_Dec_2020.csv fails to pass the quality check.
/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/2013-TM-12-December_News.csv passed the quality check.
/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/IVA_12_December_2021.csv fails to pass the quality check.
/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/2014-TM-12-December-News.csv passed the quality check.
/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/temp/IVA_6_June_2019.csv passed the quality check.
/Users/czhang/Desktop/paci

In [77]:
df["Visitors"].str.replace("-", "")

0     108808
1      89952
2      95117
3     109170
4     115634
5      10315
6       9148
7       9749
8      12283
9      10468
10      9548
11      5835
12      7026
13      9445
14      8359
15     10607
16     12612
17     12275
18     11700
19     10880
20      9860
21     12481
Name: Visitors, dtype: object

### Visitor Arrivals by Usual Country of Residence

In [178]:
bycountry_err_dict = {
    "file": [],
    "reason": []
}

for file in dec_lst:
    filepath = "/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/" + file
    print(file, locate_table(
        filepath, "Visitor Arrivals by Usual Country of Residence", ignore_case=True))
    try:
        df = load_pdf(
            filepath, "Visitor Arrivals by Usual Country of Residence", 2)
        df = df.iloc[:, :-2].dropna(thresh=4, axis=1)

        headers, row1 = df.columns.to_list(), df.iloc[0].to_list()
        newheader = list()
        for header, row in zip(headers, row1):
            if type(header) != str:
                newheader.append(str(row))
            else:
                newheader.append(str(header))

        newheader[-1] = "Total"
        newheader[newheader.index("Countries")], newheader[newheader.index(
            "nan")] = "Other PIC", "Europe"

        df.columns = newheader
        df = df.iloc[2:].reset_index().drop("index", axis=1)
        df = remove_separator(df)
        if check_quality(df, ["Month", "Year"], "Total"):
            print(f"  {file} pass the quality check.")
            saved_path = "/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/byorigin/" + \
                file.split(".")[0] + ".csv"
            df.to_csv(saved_path, encoding="utf-8")
        else:
            print(f"  {file} could have column errors")
            saved_path = "/Users/czhang/Desktop/pacific-observatory/data/tourism/vanuatu/byorigin/" + \
                file.split(".")[0] + ".csv"
            df.to_csv(saved_path, encoding="utf-8")
            bycountry_err_dict["file"].append(file)
            bycountry_err_dict["reason"].append("Column Error")
    except:
        print(f"  {file} has an error.")
        bycountry_err_dict["file"].append(file)
        bycountry_err_dict["reason"].append("Missing Error")

Tou12_December_News_2005.pdf {'table_loc': []}
  Tou12_December_News_2005.pdf could have column errors
Tou12_December_News_2004.pdf {'table_loc': []}
  Tou12_December_News_2004.pdf has an error.
Tou12_December_News_2007.pdf {'table_loc': [3]}
  Tou12_December_News_2007.pdf could have column errors
IAS_12_December_2015.pdf {'table_loc': [9]}
  IAS_12_December_2015.pdf pass the quality check.
2012-TM-12-December_News.pdf {'table_loc': [8]}
  2012-TM-12-December_News.pdf could have column errors
IAS_12_December_2016.pdf {'table_loc': [9]}
  IAS_12_December_2016.pdf pass the quality check.
IVA_12_December_2021.pdf {'table_loc': [6]}
  IVA_12_December_2021.pdf pass the quality check.
2014-TM-12-December-News.pdf {'table_loc': [7]}
  2014-TM-12-December-News.pdf pass the quality check.
2011-TM-12-December_News.pdf {'table_loc': [6]}
  2011-TM-12-December_News.pdf could have column errors
2013-TM-12-December_News.pdf {'table_loc': [7]}
  2013-TM-12-December_News.pdf could have column errors
T

In [179]:
pd.DataFrame(bycountry_err_dict)

,file,reason
0,Tou12_December_News_2005.pdf,Column Error
1,Tou12_December_News_2004.pdf,Missing Error
2,Tou12_December_News_2007.pdf,Column Error
3,2012-TM-12-December_News.pdf,Column Error
4,2011-TM-12-December_News.pdf,Column Error
5,2013-TM-12-December_News.pdf,Column Error
6,TM12_December_2009_News.pdf,Column Error
7,TM12_December_2008_News.pdf,Column Error
8,2010-TM-12-December_News.pdf,Missing Error


In [5]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser


output_string = StringIO()
with open(pdf_path, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
        
output_string.getvalue()